In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("../data/raw/survey_results.csv")
print(f"Shape for Data Frame {df.shape}")
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
dupes = df[df.duplicated(subset=["respondent_id"])]
dupes.shape


In [ ]:
dupes

In [ ]:
df.isna().sum()

#### Respondent_id is not null Let me check duplicate on this column.

In [ ]:
duplicate_respondent_Id = df['respondent_id'].duplicated().sum()
type(duplicate_respondent_Id)
duplicate_respondent_Id

In [ ]:
dupe_ids = df['respondent_id'][df['respondent_id'].duplicated()].unique()
print(dupe_ids)

dupes = df[df['respondent_id'].isin(dupe_ids)].sort_values('respondent_id')
print(dupes)


In [ ]:
df = df.drop_duplicates(subset=['respondent_id'], keep='first')


In [ ]:
print(f"Shape of Data Frame after drop duplicates {df.shape}")

#### Outliter Detection on the Age.

In [ ]:
df.age.describe()

In [ ]:
Q1 = df['age'].quantile(0.25)
Q3 = df['age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = max(0,Q1 - 1.5 * IQR)
upper_bound = Q3 + 1.5 * IQR

print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")


In [ ]:
outliers = df[(df['age'] > upper_bound) | (df['age'] < lower_bound )] 
print(outliers['age'].value_counts())


In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df['age'])
plt.title("Age Boxplot")
plt.ylabel("Age")
plt.show()

df['age'].hist(bins=30)
plt.title("Age Distribution")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()


In [ ]:
df.shape

In [ ]:
df_age_above_70 = df[df['age'] > 70]
df_age_above_70.shape

In [ ]:
df_clean = df[~(df['age'] >70)]
df_clean.shape


In [ ]:
df_clean.isna().sum()

In [ ]:
mode_consume_frequency = df_clean['consume_frequency(weekly)'].mode()[0]
mode_consume_frequency

In [ ]:
mode_purchase_channel = df_clean['purchase_channel'].mode()[0]
mode_purchase_channel

In [ ]:
df_clean['consume_frequency(weekly)']= df_clean['consume_frequency(weekly)'].fillna(mode_consume_frequency)


In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean['purchase_channel'] = df_clean['purchase_channel'].fillna(mode_purchase_channel)

In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean['income_levels'] = df_clean['income_levels'].fillna('Not Reported')

In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean['zone'].unique()

In [ ]:
df_clean['current_brand'].unique()

In [ ]:
df_clean['zone'].value_counts()

In [ ]:
df_clean['current_brand'].value_counts()

In [ ]:
zone_correction = {
    "Metor":"Metro", "urbna":"Urban"
}
df_clean['zone'] = df_clean['zone'].replace(zone_correction)

In [ ]:
df_clean['zone'].unique()

In [ ]:
current_brand_correction = {
    "newcomer":"Newcomer", "Establishd":"Established"
}
df_clean['current_brand'] = df_clean['current_brand'].replace(current_brand_correction)

In [ ]:
df_clean['current_brand'].unique()

In [ ]:
def assign_age_group(age):

    if 18 <= age <=25 :
        return "18-25"
    elif 26 <= age <= 35 :
        return "26-35"
    elif 36 <= age <=45 :
        return "36-45"
    elif 46 <= age <= 55 :
        return "46-55"
    elif 56 <= age <=70:
        return "56-70"
    else:
        return "70+"

# Apply function
df_clean['age_group'] = df_clean['age'].apply(assign_age_group)

cat_order = ["18-25","26-35","36-45","46-55","56-70","70+"]

df_clean["age_group"] = pd.Categorical(df_clean['age_group'],
                                       categories=cat_order,ordered=True)

In [ ]:

df_clean = df_clean.drop(columns=['age'])

In [ ]:
df_clean.info()

In [ ]:
freq_map = {
    "0-2 times":1,
    "3-4 times":2,
    "5-7 times":3
}

aware_map = {
    "0 to 1": 1,
    "2 to 4":2,
    "above 4":3
}

def calculate_cf_ab(row):
    f = freq_map.get(row['consume_frequency(weekly)'], 0)   # default 0 if not found
    a = aware_map.get(row['awareness_of_other_brands'], 0)
    
    if a == 0:   # prevent division by zero
        return None
    
    score = f / (a + f)
    return round(score, 2)


df_clean['cf_ab_score'] = df_clean.apply(calculate_cf_ab, axis=1)

In [ ]:
df_clean['cf_ab_score'].describe()

In [ ]:
zone_map = {
    "Urban": 3, "Metro": 4,"Rural": 1,"Semi-Urban": 2
}

income_level_map = {
     "<10L": 1,"10L - 15L": 2,"16L - 25L": 3, "26L - 35L": 4,"> 35L": 5,"Not Reported": 0
}

def calculate_zas_score(row):

    zone_score = zone_map.get(row['zone'],1)
    
    income_score = income_level_map.get(row['income_levels'],1)

    zas_score = zone_score * income_score

    return zas_score

In [ ]:
df_clean['zas_score'] = df_clean.apply(calculate_zas_score,axis=1)

In [ ]:
# (1) Optional: make key text columns categorical to save memory & speed comparisons
cat_cols = ['current_brand', 'reasons_for_choosing_brands']
for c in cat_cols:
    df_clean[c] = df_clean[c].astype('category')

# (2) Vectorized binary indicator:
# bsi = 1 if current_brand != "Established" AND reason in {"Price","Quality"}
mask = (
    (df_clean['current_brand'] != "Established") &
    (df_clean['reasons_for_choosing_brands'].isin(["Price", "Quality"]))
)
df_clean['bsi'] = mask.astype('int8')   # int8 saves memory vs int64


In [ ]:
df_clean['age_group'].value_counts()

In [ ]:
import pandas as pd

pd.crosstab(df_clean['age_group'], df_clean['occupation'])


In [ ]:
df_clean = df_clean[~((df_clean['age_group'] == '56-70') & 
                      (df_clean['occupation'] == 'Student'))]


In [ ]:
pd.crosstab(df_clean['age_group'], df_clean['occupation'])

In [ ]:
df_clean.info()

In [ ]:
df_clean['cf_ab_score'].describe()

In [ ]:
unique_record = df_clean['zas_score'].unique()
unique_record.size

In [ ]:
rows = df_clean.shape[0]   # gives row count only
print(rows) 

In [ ]:
rows = len(df_clean)
print(rows)


In [ ]:
df_bsi_zero = df_clean[df_clean['bsi'] == 0]
df_bsi_zero.shape[0]

In [ ]:
df_clean.head()

In [ ]:
df_clean.info()

In [ ]:
df_clean.to_csv("../data/survey_cleaned.csv", index=False)